In [5]:
# Text Generation with Markov Chain and Random Walk

## Import the necessary Libraries

import numpy as np
import os
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Read the Data

Read all Sherlock Holmes stories.

In [2]:
!unzip Sherlock.zip

Archive:  Sherlock.zip
   creating: Sherlock/
   creating: Sherlock/sherlock/
   creating: Sherlock/sherlock/sherlock/
  inflating: Sherlock/sherlock/sherlock/3gab.txt  
  inflating: Sherlock/sherlock/sherlock/3gar.txt  
  inflating: Sherlock/sherlock/sherlock/3stu.txt  
  inflating: Sherlock/sherlock/sherlock/abbe.txt  
  inflating: Sherlock/sherlock/sherlock/advs.txt  
  inflating: Sherlock/sherlock/sherlock/bery.txt  
  inflating: Sherlock/sherlock/sherlock/blac.txt  
  inflating: Sherlock/sherlock/sherlock/blan.txt  
  inflating: Sherlock/sherlock/sherlock/blue.txt  
  inflating: Sherlock/sherlock/sherlock/bosc.txt  
  inflating: Sherlock/sherlock/sherlock/bruc.txt  
  inflating: Sherlock/sherlock/sherlock/cano.txt  
  inflating: Sherlock/sherlock/sherlock/card.txt  
  inflating: Sherlock/sherlock/sherlock/case.txt  
  inflating: Sherlock/sherlock/sherlock/chas.txt  
  inflating: Sherlock/sherlock/sherlock/cnus.txt  
  inflating: Sherlock/sherlock/sherlock/copp.txt  
  inflating: S

## Read all the stories

In [6]:

story_path = './Sherlock/sherlock/sherlock/'

def read_all_stories (story_path):
    txt = []

    for _,_, files in os.walk(story_path):
        for file in files:
            with open(story_path+file) as f:
                for line in f:
                    line = line.strip()
                    if line=='----------': break
                    if line!='': txt.append(line)
    return txt

In [7]:
stories = read_all_stories(story_path)

In [8]:
print('Number of lines =', len(stories))

Number of lines = 215021


## Cleaning the Text

In [9]:
def clean_txt(txt):
    cleaned_txt = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt+=words
    return cleaned_txt

In [10]:
cleaned_stories = clean_txt(stories)

In [11]:
print('Number of words = ',  len(cleaned_stories))

Number of words =  2332247


In [13]:
cleaned_stories

['the',
 'gloria',
 'scott',
 'arthur',
 'conan',
 'doyle',
 'i',
 'have',
 'some',
 'papers',
 'here',
 'said',
 'my',
 'friend',
 'sherlock',
 'holmes',
 'as',
 'we',
 'sat',
 'one',
 'winters',
 'night',
 'on',
 'either',
 'side',
 'of',
 'the',
 'fire',
 'which',
 'i',
 'really',
 'think',
 'watson',
 'that',
 'it',
 'would',
 'be',
 'worth',
 'your',
 'while',
 'to',
 'glance',
 'over',
 'these',
 'are',
 'the',
 'documents',
 'in',
 'the',
 'extraordinary',
 'case',
 'of',
 'the',
 'gloria',
 'scott',
 'and',
 'this',
 'is',
 'the',
 'message',
 'which',
 'struck',
 'justice',
 'of',
 'the',
 'peace',
 'trevor',
 'dead',
 'with',
 'horror',
 'when',
 'he',
 'read',
 'it',
 'he',
 'had',
 'picked',
 'from',
 'a',
 'drawer',
 'a',
 'little',
 'tarnished',
 'cylinder',
 'and',
 'undoing',
 'the',
 'tape',
 'he',
 'handed',
 'me',
 'a',
 'short',
 'note',
 'scrawled',
 'upon',
 'a',
 'of',
 'slate',
 'the',
 'supply',
 'of',
 'game',
 'for',
 'london',
 'is',
 'going',
 'steadily',
 

# Create the Markov Model

In [14]:
def make_markov_model (cleaned_stories, n_gram=2):
    markov_model = {}

    for i in range(len(cleaned_stories)-n_gram-1):

        curr_state, next_state = "", ""

        for j in range(n_gram):
            curr_state += cleaned_stories[i+j] + " "
            next_state += cleaned_stories[i+j+n_gram] + " "

        curr_state = curr_state[:-1]
        next_state = next_state[:-1]

        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1

    # Calculate the transition probabilities

    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total


    return markov_model


In [15]:
markov_model =  make_markov_model(cleaned_stories)

In [17]:
print('Number of states = ', len(markov_model.keys()))

Number of states =  208718


In [ ]:
list(markov_model.keys())

In [20]:
list(markov_model.values())

[{'scott arthur': 0.029411764705882353,
  'scott and': 0.35294117647058826,
  'scott had': 0.11764705882352941,
  'scott in': 0.11764705882352941,
  'scott the': 0.08823529411764706,
  'scott i': 0.08823529411764706,
  'scott may': 0.11764705882352941,
  'scott he': 0.08823529411764706},
 {'arthur conan': 0.021739130434782608,
  'and this': 0.08695652173913043,
  'from her': 0.08695652173913043,
  'bound for': 0.08695652173913043,
  'had been': 0.08695652173913043,
  'in an': 0.08695652173913043,
  'and of': 0.08695652173913043,
  'was set': 0.08695652173913043,
  'the musgrave': 0.06521739130434782,
  'i have': 0.06521739130434782,
  'and my': 0.08695652173913043,
  'may serve': 0.08695652173913043,
  'he read': 0.06521739130434782},
 {'conan doyle': 1.0},
 {'doyle i': 0.10144927536231885,
  'doyle in': 0.057971014492753624,
  'doyle isa': 0.014492753623188406,
  'doyle we': 0.043478260869565216,
  'doyle the': 0.07246376811594203,
  'doyle somewhere': 0.014492753623188406,
  'doyle w

In [21]:

print("All possible transitions from 'game for' state : ")
print(markov_model['game for'])

All possible transitions from 'game for' state : 
{'london is': 0.27586206896551724, 'a morning': 0.13793103448275862, 'all it': 0.2413793103448276, 'the games': 0.13793103448275862, 'me to': 0.10344827586206896, 'a trudge': 0.10344827586206896}


In [23]:

print("All possible transitions from 'the game' state : ")
print(markov_model['the game'])

All possible transitions from 'the game' state : 
{'is up': 0.06306306306306306, 'is and': 0.036036036036036036, 'for all': 0.06306306306306306, 'is afoot': 0.036036036036036036, 'my own': 0.02702702702702703, 'at any': 0.02702702702702703, 'mr holmes': 0.02702702702702703, 'ay whats': 0.02702702702702703, 'my friend': 0.02702702702702703, 'fairly by': 0.02702702702702703, 'is not': 0.02702702702702703, 'was not': 0.02702702702702703, 'would have': 0.036036036036036036, 'in their': 0.036036036036036036, 'for the': 0.036036036036036036, 'was up': 0.09009009009009009, 'i am': 0.02702702702702703, 'now count': 0.02702702702702703, 'may wander': 0.02702702702702703, 'now a': 0.02702702702702703, 'worth it': 0.02702702702702703, 'you are': 0.02702702702702703, 'your letter': 0.02702702702702703, 'is hardly': 0.02702702702702703, 'was afoot': 0.036036036036036036, 'in that': 0.036036036036036036, 'the lack': 0.036036036036036036, 'was in': 0.02702702702702703, 'was whist': 0.0360360360360360

In [24]:

print("All possible transitions from 'my dear' state : ")
print(markov_model['my dear'])

All possible transitions from 'my dear' state : 
{'dear son': 0.0048134777376654635, 'son the': 0.0048134777376654635, 'boy is': 0.0048134777376654635, 'watson to': 0.01444043321299639, 'watson said': 0.06498194945848375, 'watson i': 0.0421179302045728, 'madam said': 0.009626955475330927, 'wife knew': 0.0048134777376654635, 'watson the': 0.01444043321299639, 'arthur i': 0.0048134777376654635, 'fellow we': 0.013237063778580024, 'watson remember': 0.0048134777376654635, 'young lady': 0.036101083032490974, 'lady you': 0.0036101083032490976, 'holmes you': 0.0036101083032490976, 'fellow no': 0.0048134777376654635, 'watson where': 0.0048134777376654635, 'holmes it': 0.0036101083032490976, 'fellow i': 0.03369434416365824, 'watson theres': 0.0048134777376654635, 'watson you': 0.036101083032490974, 'fellow that': 0.00842358604091456, 'holmes i': 0.024067388688327317, 'watson do': 0.0048134777376654635, 'watson happens': 0.0036101083032490976, 'watson but': 0.01684717208182912, 'fellow you': 0.0

# Generate Sherlock Holmes Stories

In [38]:
def generate_story(markov_model, limit=100, start='my god'):
    n = 0
    curr_state = start
    next_state = None

    story=""
    story+=curr_state+ " "

    while n<limit:
        # Random Walk through the Markov Chain based on the Transition Probabilities
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(markov_model[curr_state].values())) # choose state weighted by the transition probability

        curr_state = next_state[0]
        story+=curr_state+ " "
        n+=1

    return story


In [36]:
mylist = ['apple', 'banana', 'cherry']

random.choices(mylist, [1, 2, 1], k=10)

['banana',
 'apple',
 'banana',
 'cherry',
 'banana',
 'banana',
 'banana',
 'banana',
 'cherry',
 'banana']

In [40]:
for i in range(10):
    print(str(i) + " > "+ generate_story(markov_model, start='my business', limit=8))

0 > my business said he as he asked and her fingers were trembling as she tried to scream and 
1 > my business but he thrust it into the case of and clumsy you can think that she has 
2 > my business william kindly leave me alone what are you then what hellish thing what absolutely damning evidence 
3 > my business from there on foot than on any criminal wish to make a little investigation showed me 
4 > my business and not against me but i see that it was feet how do you think that 
5 > my business took me just as it occurred when i found that he had set himself to hunt 
6 > my business to find some remains of this world something has come away from home in a cab 
7 > my business came to baker street and had held out his hand gazed hard at the station to 
8 > my business to know that you would never have been committed late in the until i had taken 
9 > my business to follow it up somehow or another you dont find your fancy kind o good it 


In [41]:
for i in range(10):
    print(str(i) + " > "+ generate_story(markov_model, start='dear watson', limit=8))

0 > dear watson was very drunk and in a hearty meal when it fell below the high pay can 
1 > dear watson there is certainly a possible point of view its simply awful but you have yourself any 
2 > dear watson said he it is the cry of cooee which was more menacing than his companion had 
3 > dear watson we have several among our workpeople and cleaners i dare say my natural prey briefly watson 
4 > dear watson the more readily upon him what i wished to see sherlock holmes had been the of 
5 > dear watson happens to be the business manager to the and villainous pair whom we were compelled to 
6 > dear watson tickets and all expenses paid on a princely scale splendid but why keep me after what 
7 > dear watson said he swaying his face about you really are her friend stand by me in the 
8 > dear watson hence the fanciful name given by the prime minister good heavens to think that woolwich can 
9 > dear watson the two constables at the door and then afterwards they remembered us and sent it

# Check the possible transtions ffrom a given state

In [42]:
generate_story(markov_model, start='sherlock holmes', limit=8)

'sherlock holmes laughed i am afraid that i might get on to those who had first known douglas '

In [44]:
out_dict = markov_model['sherlock holmes']
len(list(out_dict.keys()))

328

In [ ]:
out_dict.keys()

In [47]:
list(out_dict.keys()).index('laughed i')

69

In [48]:
list(out_dict.keys())[69]

'laughed i'

In [49]:
generate_story(markov_model, start='sherlock holmes', limit=8)

'sherlock holmes was it after hearing the shot the candle burning he had to part i had followed '

In [50]:
list(out_dict.keys()).index('was it')

179

In [51]:
list(out_dict.keys())[179]

'was it'

In [53]:
out_dict = markov_model['was it']
out_dict


{'possible that': 0.029953917050691243,
 'said holmes': 0.009216589861751152,
 'found in': 0.018433179723502304,
 'from mr': 0.009216589861751152,
 'oh not': 0.009216589861751152,
 'the thief': 0.009216589861751152,
 'some one': 0.009216589861751152,
 'an accident': 0.009216589861751152,
 'he sank': 0.009216589861751152,
 'in this': 0.009216589861751152,
 'found next': 0.0069124423963133645,
 'caused by': 0.009216589861751152,
 'before you': 0.009216589861751152,
 'indeed a': 0.009216589861751152,
 'it was': 0.009216589861751152,
 'done he': 0.009216589861751152,
 'you wont': 0.009216589861751152,
 'you put': 0.018433179723502304,
 'done ill': 0.0069124423963133645,
 'is true': 0.0069124423963133645,
 'raised it': 0.0069124423963133645,
 'my imagination': 0.0069124423963133645,
 'after hearing': 0.0069124423963133645,
 'was the': 0.0069124423963133645,
 'need hardly': 0.0069124423963133645,
 'crime last': 0.0069124423963133645,
 'yes that': 0.016129032258064516,
 'and you': 0.006912442

In [54]:
generate_story(markov_model, start='sherlock holmes', limit=4)

'sherlock holmes gazing fixedly at my baker street with a '